# Shark League Metric 

## Import Python libraries

In [38]:
import pandas as pd
import logging
import swc_simple_client as swc

## Configure logging

In [39]:
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

logging.basicConfig(
    filename='shark_notebook.log',  
    level=logging.INFO,  
)

## Setup notebook variables

In [40]:
base_url = "[input base url]"


## Get Custom Max Scores
### Use endpoint: LIST_WEEKS_ENDPOINT 

In [ ]:
week_api_response = swc.call_api_endpoint(base_url,swc.LIST_WEEKS_ENDPOINT)
weeks_df = pd.DataFrame(week_api_response.json())
weeks_df['year'] = weeks_df['week_number'].str.slice(0, 4).astype(int)
weeks_df['week'] = weeks_df['week_number'].str.slice(4, 6).astype(int)

weeks_df = weeks_df.query('week <= 14')

max_totals_grouped_df = weeks_df.groupby('year').agg(
    ppr_12_max_points=('ppr_12_max_points', 'sum'), 
    half_ppr_8_max_points=('half_ppr_8_max_points', 'sum'))

display(max_totals_grouped_df)

## Get League Scoring Type
### Use Endpoint: LIST_LEAGUES_ENDPOINT

In [ ]:
league_api_response = swc.call_api_endpoint(base_url,swc.LIST_LEAGUES_ENDPOINT)
leagues_df = pd.DataFrame(league_api_response.json())
leagues_df = leagues_df.drop(columns=['teams','last_changed_date'])
display(leagues_df)

## Get Regular Season Scoring Totals - By Team
### Use Endpoint: LIST_TEAMS_ENDPOINT

In [ ]:
team_api_response = swc.call_api_endpoint(base_url,swc.LIST_TEAMS_ENDPOINT)

teams_parsed_df = pd.json_normalize(team_api_response.json(), 'weekly_scores', ['league_id', 'team_id', 'team_name'])

teams_parsed_df['year'] = teams_parsed_df['week_number'].str.slice(0, 4).astype(int)
teams_parsed_df['week'] = teams_parsed_df['week_number'].str.slice(4, 6).astype(int)

teams_parsed_df2 = teams_parsed_df.drop(columns=['week_number'])

#get only regular season teams
teams_regular_season_df = teams_parsed_df.query('week <= 14')

#get team season totals
team_totals_df = teams_regular_season_df.groupby(['league_id', 'team_id', 'year'], as_index = False)['fantasy_points'].sum()

team_totals_df.head()

In [ ]:
team_api_response = swc.call_api_endpoint(base_url,swc.LIST_TEAMS_ENDPOINT)

teams_parsed_df2 = pd.json_normalize(team_api_response.json(), 'weekly_scores', ['league_id', 'team_id', 'team_name'])

teams_parsed_df2['year'] = teams_parsed_df2['week_number'].str.slice(0, 4).astype(int)
teams_parsed_df2['week'] = teams_parsed_df2['week_number'].str.slice(4, 6).astype(int)

teams_parsed_df2 = teams_parsed_df2.drop(columns=['week_number'])

#get only regular season teams
teams_regular_season_df2 = teams_parsed_df2.query('week <= 14')

#get team season totals
team_totals_df2 = teams_regular_season_df2.groupby(['league_id', 'team_id', 'year'], as_index = False)['fantasy_points'].sum()

team_totals_df2.head()

## League Balance Score
### Using Coefficient of Variation (CV) of league regular season totals

In [ ]:
league_stats_df = team_totals_df.groupby(['league_id','year']).agg(
    league_points_sum=('fantasy_points', 'sum'),
    league_points_mean=('fantasy_points', 'mean'),
    league_points_stdev=('fantasy_points', 'std'),
    league_balance_score=('fantasy_points', lambda x: (100 -(x.std() / x.mean()) * 100))
).reset_index()

display(league_stats_df.sort_values(by='league_balance_score', ascending=False))

## League Juice Score
### Compare league scoring to max potential scoring

In [ ]:
league_stats_with_league_max_df = (league_stats_df[['league_id','year', 'league_points_sum','league_balance_score']]
               .merge(max_totals_grouped_df,left_on = 'year', right_on='year'))

combined_metrics_df = (leagues_df[['league_id','league_name','scoring_type', 'league_size']]
                         .merge(league_stats_with_league_max_df, left_on = 'league_id', right_on = 'league_id'))

combined_metrics_df['league_juice_score'] = combined_metrics_df.apply(
    lambda row: (
        100 * (row['league_points_sum'] / row['ppr_12_max_points'])
        if (row['scoring_type'] == 'PPR' and row['league_size'] == 12) else (
            100 * (row['league_points_sum'] / row['half_ppr_8_max_points'])
            if (row['scoring_type'] == 'Half-PPR' and row['league_size'] == 8) else None
        )
    ),
    axis=1
)

combined_metrics_df = (combined_metrics_df.drop(
    columns=['scoring_type','league_size','league_points_sum','ppr_12_max_points','half_ppr_8_max_points',] )
)
display(combined_metrics_df)

# Create Shark League Score
## Shark League Score = (2 * League Juice Score) + League Balance Score

In [ ]:
combined_metrics_df['shark_league_score'] = combined_metrics_df.apply(
    lambda league: (2 * league['league_juice_score']) +  league['league_balance_score'],
    axis=1
)
display(combined_metrics_df.sort_values(by='shark_league_score', ascending=False))